In [1]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

## @params: [JOB_NAME]

glueContext = GlueContext(sc)
spark = glueContext.spark_session
## @type: DataSource
## @args: [database = "forjdbc", table_name = "postgres_postgres_public_tb", transformation_ctx = "datasource0"]
## @return: datasource0
## @inputs: []
datasource0 = glueContext.create_dynamic_frame.from_catalog(database = "forjdbc", table_name = "post_gres_postgres_public_tbf", transformation_ctx = "datasource0")
datasource0.printSchema()
datasource0.toDF().show()

Starting Spark application


SparkSession available as 'spark'.
root
|-- id: int
|-- e_type: string
|-- metadata: string

+---+------+------------+
| id|e_type|    metadata|
+---+------+------------+
|  1|  thor|{"k1":"v11"}|
|  2|  hulk|{"k2":"v12"}|
|  3|  thor|{"k1":"v13"}|
|  4|  hulk|{"k1":"v14"}|
|  5|  hulk|{"p1":"v14"}|
+---+------+------------+

In [2]:
output = SplitRows.apply(frame = datasource0, comparison_dict = {"e_type": {"=": "thor"}}, name1="frame1", name2="frame2",transformation_ctx = "output")

frame1= output.select("frame1")
frame2 = output.select("frame2")

print("frame1")
frame1.toDF().show()

print("frame2")
frame2.toDF().show()

frame1
+---+------+------------+
| id|e_type|    metadata|
+---+------+------------+
|  1|  thor|{"k1":"v11"}|
|  3|  thor|{"k1":"v13"}|
+---+------+------------+

frame2
+---+------+------------+
| id|e_type|    metadata|
+---+------+------------+
|  2|  hulk|{"k2":"v12"}|
|  4|  hulk|{"k1":"v14"}|
|  5|  hulk|{"p1":"v14"}|
+---+------+------------+

In [3]:
unboxedFrame1 = Unbox.apply(frame = frame2,path ="metadata",format = "json", withHeader = "True")
print("unboxedFrame1")
unboxedFrame1.printSchema()
unboxedFrame1.toDF().show()

afterRelationlize1 = Relationalize.apply(frame = unboxedFrame1, staging_path = "s3://kanwarsh/GlueTempdir/", name = "finaldata", transformation_ctx = "afterRelationlize1")
y = afterRelationlize1.select("finaldata")
print("AfterRelationlizeFrame1")
y.printSchema()
y.toDF().show()

unboxedFrame1
root
|-- id: int
|-- e_type: string
|-- metadata: struct
|    |-- k2: string
|    |-- k1: string
|    |-- p1: string

+---+------+---------------+
| id|e_type|       metadata|
+---+------+---------------+
|  2|  hulk|[v12,null,null]|
|  4|  hulk|[null,v14,null]|
|  5|  hulk|[null,null,v14]|
+---+------+---------------+

AfterRelationlizeFrame1
root
|-- id: int
|-- e_type: string
|-- metadata.p1: string
|-- metadata.k2: string
|-- metadata.k1: string

+---+------+-----------+-----------+-----------+
| id|e_type|metadata.p1|metadata.k2|metadata.k1|
+---+------+-----------+-----------+-----------+
|  2|  hulk|       null|        v12|       null|
|  4|  hulk|       null|       null|        v14|
|  5|  hulk|        v14|       null|       null|
+---+------+-----------+-----------+-----------+